# 2. Create Your Own Agents

This notebook expects that you're all setup (Have Claude desktop app running with an MCP Server setup in the previous notebook).

Here we'll show you **how to pass your own Python functions to Claude** 

In [2]:
import fused
import json
import os
import time
from pathlib import Path

In [3]:
# We still need your local paths
PATH_TO_CLAUDE_CONFIG = f"{str(Path.home())}/Library/Application Support/Claude/claude_desktop_config.json"


if not os.path.exists(PATH_TO_CLAUDE_CONFIG):
    # Creating the config file
    os.makedirs(os.path.dirname(PATH_TO_CLAUDE_CONFIG), exist_ok=True)
    with open(PATH_TO_CLAUDE_CONFIG, 'w') as f:
        json.dump({}, f)

assert os.path.exists(PATH_TO_CLAUDE_CONFIG), "Please update the PATH_TO_CLAUDE_CONFIG variable with the correct path to your Claude config file"

In [4]:
# Local path to the Claude app
CLAUDE_APP_PATH = "/Applications/Claude.app"
assert os.path.exists(CLAUDE_APP_PATH), "Please update the CLAUDE_APP_PATH variable with the correct path to your Claude app"

In [5]:
# Change this path if you're not running this from the repo root
WORKING_DIR = os.getcwd()

In [22]:
# We'll load the commons folder once again to have our helper functions
commit = "28821ea"
common = fused.load(f"https://github.com/fusedio/udfs/tree/{commit}/public/common").utils

In [7]:
# And see which agents we have available
json.load(open(os.path.join(WORKING_DIR, "agents.json")))

{'agents': [{'name': 'get_current_time', 'udfs': ['current_utc_time']},
  {'name': 'fused_docs', 'udfs': ['list_public_udfs', 'reading_fused_docs']}]}

We'll make a simple UDF that gives us the top 5 stories from Hacker News as our basis

In [9]:
@fused.udf
def udf(story_type: str = "top"):
    """
    Fetches top posts from Hacker News as a dataframe.
    
    Parameters:
    story_type (str): Type of stories to fetch. Options are:
                      - "top" for top stories
                      - "newest" for latest stories
    
    Returns:
    pandas.DataFrame: DataFrame containing HN posts with columns:
                     id, title, url, score, by (author), time, descendants (comments)
    """
    import pandas as pd
    import requests
    import time
    from datetime import datetime
    
    # Validate input
    if story_type not in ["top", "newest"]:
        raise ValueError('Invalid story_type. Must be "top" or "newest"')
    
    # Map story_type to the appropriate HN API endpoint
    endpoint_map = {
        "top": "topstories",
        "newest": "newstories"
    }
    
    endpoint = endpoint_map[story_type]
    
    # Fetch the list of top or newest story IDs
    response = requests.get(f"https://hacker-news.firebaseio.com/v0/{endpoint}.json")
    story_ids = response.json()
    
    # Only doing 5 stories for now
    story_ids = story_ids[:5]
    
    # Fetch details for each story ID
    stories = []
    for story_id in story_ids:
        try:
            # Get the story details
            story_response = requests.get(f"https://hacker-news.firebaseio.com/v0/item/{story_id}.json")
            story = story_response.json()
            
            # Skip if not a story or missing key fields
            if not story or story.get('type') != 'story' or 'title' not in story:
                continue
                
            # Add to our list
            stories.append({
                'id': story.get('id'),
                'title': story.get('title'),
                'url': story.get('url', ''),
                'score': story.get('score', 0),
                'by': story.get('by', ''),
                'time': datetime.fromtimestamp(story.get('time', 0)),
                'descendants': story.get('descendants', 0)
            })
            
            # Brief pause to avoid overloading the API
            time.sleep(0.1)
            
        except Exception as e:
            print(f"Error fetching story {story_id}: {e}")
    
    # Convert the list of stories to a DataFrame
    df = pd.DataFrame(stories)
    
    # Add a timestamp for when the data was fetched
    df['fetched_at'] = datetime.now()
    print(f"{df=}")
    
    return df

In [10]:
# We can run this UDF locally with `fused.run(udf)`
fused.run(udf)

,id,title,url,score,by,time,descendants,fetched_at
0,43410061,The Lost Art of Research as Leisure,https://kasurian.com/p/research-as-leisure,166,altilunium,2025-03-19 10:09:15,77,2025-03-19 13:25:12.563609
1,43400989,Two new PebbleOS watches,https://ericmigi.com/blog/introducing-two-new-...,1441,griffinli,2025-03-18 15:59:27,428,2025-03-19 13:25:12.563609
2,43406710,Make Ubuntu packages 90% faster by rebuilding ...,https://gist.github.com/jwbee/7e8b27e298de8bbb...,446,jeffbee,2025-03-18 23:55:17,258,2025-03-19 13:25:12.563609
3,43401245,Apple restricts Pebble from being awesome with...,https://ericmigi.com/blog/apple-restricts-pebb...,1556,griffinli,2025-03-18 16:23:21,958,2025-03-19 13:25:12.563609
4,43410885,The Origin of the Pork Taboo,https://archaeology.org/issues/march-april-202...,3,diodorus,2025-03-19 12:16:06,0,2025-03-19 13:25:12.563609


## Creating our own agent with our own UDF

We want to be able to give this over to Claude we need to create a new agent for. We'll need a few things:
- `agent_name` -> The name of the agent we want to call. This `agent_name` can call multiple `udf_names`
- `udf_name` -> The name of our UDF. Right now we defined it as `udf`, but we can name it for example `recent_hacker_news_stories`
- `mcp_metadata` -> We also need to give some info to Claude about what our UDF does, and what parameters it have access to. This includes:
    - `description`: What the UDF does
    - `parameters`: A dict containing:
        - `"name"` -> The name of the parameter. In our example above this would be `story_type`
        - `"type"` -> The python type of the parameter. In our example it would be `str`

In [13]:
hacker_news_mcp_metadata = {
    "description": "This UDF gets the latest 5 stories names, url, author & time from Hacker News. It get either get the 'top' or 'newest' stories",
    "parameters": [{
        "name": "story_type",
        "type": "str",
    }]
}

hacker_news_mcp_metadata

{'description': "This UDF gets the latest 5 stories names, url, author & time from Hacker News. It get either get the 'top' or 'newest' stories",
 'parameters': [{'name': 'story_type', 'type': 'int'}]}

In [23]:
# We'll create a new Agent that can access the news
common.save_to_agent(
    agent_json_path = os.path.join(WORKING_DIR, "agents.json"),
    udf = udf,
    agent_name = "getting_the_news",
    udf_name = "recent_hacker_news_stories",
    mcp_metadata = hacker_news_mcp_metadata,
)

In [29]:
# let's look at the existing agents
agents = json.load(open(os.path.join(WORKING_DIR, "agents.json")))
print(json.dumps(agents, indent=4, sort_keys=True))

{
    "agents": [
        {
            "name": "get_current_time",
            "udfs": [
                "current_utc_time"
            ]
        },
        {
            "name": "fused_docs",
            "udfs": [
                "list_public_udfs",
                "reading_fused_docs"
            ]
        },
        {
            "name": "getting_the_news",
            "udfs": [
                "recent_hacker_news_stories"
            ]
        }
    ]
}


We've made a utility that lets you pass multiple agents to Claude if you want, each with individual UDFs

For now let's simply use the `getting_the_news` agent we created:

In [31]:
agents_list = ["getting_the_news"]

In [32]:
# Finally, we can select which Agent we want to pass to Claude in our MCP server config
common.generate_local_mcp_config(
    config_path=PATH_TO_CLAUDE_CONFIG,
    agents_list = agents_list,
    repo_path= WORKING_DIR,
)

Claude uses a specific config (that you passed under `PATH_TO_CLAUDE_CONFIG`) to know what to run under the hood. This is what we're editing for you each time you change the agent you want to run

In [33]:
# Let's read this Claude Desktop config to see what we're passing
claude_config = json.load(open(PATH_TO_CLAUDE_CONFIG))
print(json.dumps(claude_config, indent=4, sort_keys=True))

{
    "mcpServers": {
        "getting_the_news": {
            "args": [
                "run",
                "--directory",
                "/Users/maximelenormand/Library/CloudStorage/Dropbox/Mac/Documents/repos/fused-mcp",
                "main.py",
                "--runtime=local",
                "--udf-names=recent_hacker_news_stories",
                "--name=getting_the_news"
            ],
            "command": "uv"
        }
    }
}


## Now let's restart Claude with this new agent!

In [34]:
def restart_claude(claude_path: str = CLAUDE_APP_PATH):
    app_name = claude_path.split("/")[-1]

    try:
        os.system(f"pkill -f '{app_name}'")
        print(f"Killed {app_name}")
        time.sleep(2)  # Wait for shutdown
    except Exception as e:
        print(f"Claude wasn't running, so no need to kill it")

    print(f"Restarting {app_name}")
    os.system(f"open -a '{claude_path}'")  # Restart Claude

In [35]:
restart_claude()

Killed Claude.app
Restarting Claude.app
